<a href="https://colab.research.google.com/github/umslengineering/EE1108/blob/main/EE1108_chess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#chess.py

In [ ]:
# chess.py
# Full chess game with rules, timer, winner, and piece sprites

BOARD_SIZE = 8
SQUARE = 80

WIDTH = BOARD_SIZE * SQUARE
HEIGHT = BOARD_SIZE * SQUARE + 60

LIGHT = (240, 217, 181)
DARK = (181, 136, 99)

turn = "white"
selected = None

white_time = 300  # 5 minutes
black_time = 300

game_over = False
winner = ""

# -----------------------------------
# Board setup
# -----------------------------------
board = [
    ["br","bn","bb","bq","bk","bb","bn","br"],
    ["bp"]*8,
    [".."]*8,
    [".."]*8,
    [".."]*8,
    [".."]*8,
    ["wp"]*8,
    ["wr","wn","wb","wq","wk","wb","wn","wr"]
]

# Load piece images
piece_images = {} # dict
for color in ["w","b"]:
    for ptype in ["p","r","n","b","q","k"]:
        key = color+ptype
        piece_images[key] = key

# -----------------------------------
# Draw board
# -----------------------------------

def draw_row(row_index, c1, c2):
    y = row_index * SQUARE
    for col in range(BOARD_SIZE):
        x = col * SQUARE
        color = c1 if col % 2 == 0 else c2
        rect = Rect(x, y, SQUARE, SQUARE)
        screen.draw.filled_rect(rect, color)

def draw_board():
    for row in range(BOARD_SIZE):
        if row % 2 == 0:
            draw_row(row, LIGHT, DARK)
        else:
            draw_row(row, DARK, LIGHT)

# -----------------------------------
# Draw pieces
# -----------------------------------

def draw_pieces():
    for r in range(BOARD_SIZE): # r is 0,1,2,3,4,5,6,7
        for c in range(BOARD_SIZE):
            piece = board[r][c]
            if piece != "..":
                screen.blit(piece, (c*SQUARE, r*SQUARE))

## Notice that: Screen coordinates use (x, y)
## x = horizontal = column, y = vertical = row

# -----------------------------------
# Draw selection
# -----------------------------------

def draw_selection():
    if selected:
        r, c = selected
        rect = Rect(c*SQUARE, r*SQUARE, SQUARE, SQUARE)
        screen.draw.rect(rect, (0,255,0)) ## highlight color green

# -----------------------------------
# Chess Logic
	# •	sr = start row
	# •	sc = start column
	# •	tr = target row
	# •	tc = target column
  #When determining movement direction on the board,
  # the values of step_r (row step) and step_c (column step)
  # indicate how the position changes at each step along the path.
  #1. moving to the right means there is no change in rows but an increase in columns,
  #so step_r = 0 and step_c = 1.
  #2. Moving to the left also keeps the row the same but decreases the column,
  #giving step_r = 0 and step_c = -1.
  #3. Moving down increases the row while keeping the column unchanged,
  # so step_r = 1 and step_c = 0,
  #4. moving up decreases the row with no column change,
  #resulting in step_r = -1 and step_c = 0.
  #5. For a diagonal movement such as down-right,
  # both the row and column increase together,
  # so step_r = 1 and step_c = 1.
  # These step values allow the program to advance square-by-square in the correct direction
  # when checking movement paths.
# -----------------------------------

def path_clear(sr, sc, tr, tc):
    dr = tr - sr
    dc = tc - sc
    steps = max(abs(dr), abs(dc)) # number of squares moved = largest distance
    step_r = (dr // steps) if dr != 0 else 0 #Possible outcomes: 1→moving down; -1→moving up; 0→no vertical movement
    step_c = (dc // steps) if dc != 0 else 0

    for i in range(1, steps): #check squares between start and end, skip destination square
        r = sr + i*step_r    #Compute intermediate square, walks along the path.
        c = sc + i*step_c
        if board[r][c] != "..": #if square contains a piece.
            return False. #Path is blocked → move not allowed.
    return True

#Can the piece at (start row, start column) reach the target square using its normal movement pattern
def can_piece_reach(sr, sc, tr, tc):
    piece = board[sr][sc]
    if piece == "..":
        return False

    p = piece[1] #strip out the color information w/b, and leaving only the chesspiece type
    dr = tr - sr #Calculate movement distance
    dc = tc - sc

    if p == "p":
        color = piece[0] #Get pawn color
        direction = -1 if color == "w" else 1 #White moves upward (row decreases), Black moves downward (row increases).
        return abs(dc) == 1 and dr == direction

    if p == "r":
        return (sr == tr or sc == tc) and path_clear(sr, sc, tr, tc)

    if p == "b":
        return abs(dr) == abs(dc) and path_clear(sr, sc, tr, tc)

    if p == "q":
        return (sr == tr or sc == tc or abs(dr) == abs(dc)) and path_clear(sr, sc, tr, tc)

    if p == "n":
        return (abs(dr), abs(dc)) in [(2,1),(1,2)]

    if p == "k":
        return max(abs(dr), abs(dc)) == 1

    return False

def is_square_attacked(row, col, by_color): #Is the square at (row, col) being attacked by any piece of a specific color?
    for r in range(BOARD_SIZE):
        for c in range(BOARD_SIZE):
            piece = board[r][c]
            if piece != ".." and piece[0] == by_color:
              #by_color → which side is attacking, attacked by "by_color" pieces
                if can_piece_reach(r,c,row,col):
                  #Can this piece move to the target square according to its movement rules
                    return True. #YES → square is attacked
    return False #NO pieces → square is safe

def find_king(color):
    king_piece = ("w" if color=="white" else "b") + "k"
    for r in range(BOARD_SIZE):
        for c in range(BOARD_SIZE):
            if board[r][c] == king_piece:
                return (r,c)
    return None

def is_in_check(color):
    king_pos = find_king(color)
    if king_pos is None:
        return False
    opponent = "b" if color == "white" else "w"
    return is_square_attacked(king_pos[0], king_pos[1], opponent)

def would_be_in_check(sr, sc, tr, tc, color):
  # Checks whether a move would leave your king in check before actually making the move permanently
  #color = which player we are checking (e.g., "w" or "b")

    original_piece = board[tr][tc]
    #The destination square may contain a piece (capture). Save original piece at destination
    board[tr][tc] = board[sr][sc] #Moves the piece to target square.
    board[sr][sc] = ".." #Clears the starting square.

    in_check = is_in_check(color) # Check if king is in check

    #Undo the move, Restore original board state:
    board[sr][sc] = board[tr][tc]
    board[tr][tc] = original_piece
    #ensure no permanent changes were made.

    return in_check

def valid_move(sr, sc, tr, tc):
    piece = board[sr][sc]
    target = board[tr][tc]

    if piece == "..":
        return False

    color = piece[0]
    p = piece[1]

    if target != ".." and target[0] == color: # If the destination square contains your own piece, move is illegal
        return False

    if target != ".." and target[1] == "k": #If target square contains a king: move is illegal
        return False

    dr = tr - sr
    dc = tc - sc
    is_legal = False

    if p == "p":
        #	White pawns move upward → row decreases → -1
	      # Black pawns move downward → row increases → +1
        direction = -1 if color=="w" else 1
        #	white start row = 6, black start row = 1
        start_row = 6 if color=="w" else 1

        #	dr = tr - sr → row movement (vertical)
	      # dc = tc - sc → column movement (horizontal)
        # dc==0 pawn moves straight (no sideways)
        if dc==0 and dr==direction and target=="..":
            is_legal=True
        elif dc==0 and sr==start_row and dr==2*direction...
         and board[sr+direction][sc]==".." and target=="..":
            is_legal=True
        elif abs(dc)==1 and dr==direction and target!="..":
            is_legal=True #Move exactly one square, move forward one row, there must be a piece to capture.

    elif p == "r":
        if (sr==tr or sc==tc) and path_clear(sr,sc,tr,tc):
            is_legal=True

    elif p == "b":
        if abs(dr)==abs(dc) and path_clear(sr,sc,tr,tc):
            is_legal=True

    elif p == "q":
        if (sr==tr or sc==tc or abs(dr)==abs(dc)) and path_clear(sr,sc,tr,tc):
            is_legal=True

    elif p == "n":
        if (abs(dr),abs(dc)) in [(2,1),(1,2)]:
            is_legal=True

    elif p == "k":
        opponent = "b" if color=="w" else "w"
        if max(abs(dr),abs(dc))==1 and not is_square_attacked(tr,tc,opponent):
            is_legal=True #King can move exactly one square in any direction.

    if not is_legal:
        return False

    current_color = "white" if color=="w" else "black"
    if would_be_in_check(sr,sc,tr,tc,current_color):
        return False

    return True

def has_legal_moves(color):
    for sr in range(BOARD_SIZE):
        for sc in range(BOARD_SIZE):
            piece = board[sr][sc]
            if piece!=".." and ((color=="white" and piece[0]=="w") or (color=="black" and piece[0]=="b")):
                for tr in range(BOARD_SIZE):
                    for tc in range(BOARD_SIZE):
                        if valid_move(sr,sc,tr,tc):
                            return True
    return False

# -----------------------------------
# Mouse input
# -----------------------------------

def on_mouse_down(pos):
    global selected, turn, game_over, winner

    if game_over:
        return

    #	pos = mouse click position in pixels (x, y)
	  #SQUARE = size of one board square in pixels
	  #// = integer division → gives board column and row
    c = int(pos[0]//SQUARE)
    r = int(pos[1]//SQUARE)

    #Check if a piece is currently selected
    if selected is None:
        piece = board[r][c] #	Retrieves the piece at the clicked square
        if piece!=".." and ((turn=="white" and piece[0]=="w") or (turn=="black" and piece[0]=="b")):
            selected=(r,c)
    else:
        sr,sc = selected
        if valid_move(sr,sc,r,c): #Checks if moving selected piece to target is legal

            # Move piece
            board[r][c] = board[sr][sc]
            board[sr][sc] = ".."

            # Pawn Promotion
            # •	White pawn reaching row 0 → becomes queen ("wq")
	          # •	Black pawn reaching row 7 → becomes queen ("bq")
            moved_piece = board[r][c]
            if moved_piece == "wp" and r == 0:
                board[r][c] = "wq"
            elif moved_piece == "bp" and r == 7:
                board[r][c] = "bq"
            # after the move, Switch turns
            turn = "black" if turn=="white" else "white"

            if not has_legal_moves(turn): #If player cannot make any legal move → game over
                game_over=True
                if is_in_check(turn):
                    winner = "black" if turn=="white" else "white"
                else:
                    winner="draw"
                # Check if king is in check:
	              #	True → checkmate → opponent wins,	False → stalemate → draw

        selected=None #After a move or invalid click, clear selection

# -----------------------------------
# Timer update
# -----------------------------------

def update(): # update() is called 60 times per second
    global white_time, black_time, game_over, winner

    if game_over:
        return

    if turn=="white":
        white_time-=1/60
        if white_time<=0:
            winner="black"
            game_over=True
    else:
        black_time-=1/60
        if black_time<=0:
            winner="white"
            game_over=True

# -----------------------------------
# Draw everything
# -----------------------------------

def draw():
    screen.clear()
    draw_board()
    draw_selection()
    draw_pieces()

    # Shows whose turn it is (white or black) at the bottom-left.
    # (10, HEIGHT-55) → x,y coordinates on screen.
	  # fontsize=30, color="white" → makes it visible.
    screen.draw.text(f"Turn: {turn}", (10, HEIGHT-55), fontsize=30, color="white")

    #	Shows remaining white and black time at the bottom.
	  # Converts timer floats to integers using int().
	  # Positioned slightly to the right (x=200) of the turn text.
    screen.draw.text(f"W:{int(white_time)}  B:{int(black_time)}", (200, HEIGHT-55), fontsize=30, color="white")

    if not game_over and is_in_check(turn):
        screen.draw.text("CHECK!", (WIDTH-150, HEIGHT-55), fontsize=35, color="red")

    if game_over:
        if winner=="draw":
            screen.draw.text("STALEMATE!", center=(WIDTH/2, HEIGHT-30), fontsize=40, color="yellow")
        else:
            screen.draw.text(f"{winner.upper()} WINS!", center=(WIDTH/2, HEIGHT-30), fontsize=40, color="yellow")

  # Draw stalemate
	# •	If winner=="draw", display "STALEMATE!" in yellow at the center bottom.
	# •	center=(WIDTH/2, HEIGHT-30) → horizontally centered, near bottom.
	# •	Larger font: 40.

  # Draw win message
	# •	Otherwise, display WHITE WINS! or BLACK WINS!
	# •	Uses .upper() to capitalize color name.
	# •	Same styling as stalemate.